In [49]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
from IPython import display
load_dotenv()
import json
import enum


import sys

from datetime import datetime, timedelta
from sqlalchemy import create_engine, Table, Column, ForeignKey, Integer, String, DateTime, Boolean, ARRAY, DATE, FLOAT, Enum
from sqlalchemy import select, insert, within_group, func, and_, or_, delete, types, cast, Numeric
from sqlalchemy.orm import declarative_base, relationship, Session
import mysql.connector as mysql
import sqlalchemy
import pandas as pd
from fredapi import Fred
import wbgapi as wb

In [2]:
fred = Fred(api_key=os.environ['FRED_KEY'])

In [40]:
data = fred.get_series('SP500')

In [55]:
fred.get_series("IHLCHGUS")

2020-02-01     0.0
2020-02-02     0.0
2020-02-03     0.2
2020-02-04     0.3
2020-02-05     0.4
              ... 
2022-03-28    57.9
2022-03-29    57.9
2022-03-30    58.2
2022-03-31    58.3
2022-04-01    58.1
Length: 791, dtype: float64

In [58]:
fred.get_series_info("IHLCHGNEWUS")

id                                                                 IHLCHGNEWUS
realtime_start                                                      2022-04-13
realtime_end                                                        2022-04-13
title                          New Job Postings on Indeed in the United States
observation_start                                                   2020-02-01
observation_end                                                     2022-04-01
frequency                                                         Daily, 7-Day
frequency_short                                                              D
units                                         Percent Change from Feb 01, 2020
units_short                                           % Chg. from Feb 01, 2020
seasonal_adjustment                                        Seasonally Adjusted
seasonal_adjustment_short                                                   SA
last_updated                                        

In [3]:
Base = declarative_base()

class Denominations(enum.Enum):
    k = 1
    m = 2
    b = 3

class RealGDP(Base):
    __tablename__ = "real_gdp"
    id = Column(Integer, primary_key=True)
    date = Column(DATE)
    currency = Column(String(3))
    value = Column(FLOAT)
    unit = Column(Enum(Denominations))
    interval = Column(String(1))

class Treasury_Yield(Base):
    __tablename__ = "sov_yc"
    id = Column(Integer, primary_key=True)
    maturity = Column(String(10))
    date = Column(DATE)
    rate = Column(FLOAT)
    currency = Column(String(3))



In [4]:
engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER']}:{os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" )


In [8]:
Base.metadata.create_all(engine)

In [18]:
output = json.loads(r.text)

In [126]:
with Session(engine) as session:
    session.execute(delete(Treasury_Yield).where(Treasury_Yield.maturity == '1y'))
    session.commit()

In [18]:
alias_2y = select(Treasury_Yield.date, Treasury_Yield.rate).where(Treasury_Yield.maturity == '2y').alias()
alias_10y = select(Treasury_Yield.date, Treasury_Yield.rate).where(Treasury_Yield.maturity == '10y').alias()

In [132]:
print(session.execute(select(Treasury_Yield.maturity, func.count(Treasury_Yield.rate)).group_by(Treasury_Yield.maturity)).all())

[('10y', 15055), ('1y', 13988), ('2y', 11466), ('30y', 11285), ('3m', 10153), ('5y', 15055), ('FF', 24757)]


In [30]:
select(alias_10y.c.date,alias_10y.c.rate-alias_2y.c.rate).join_from(alias_10y, alias_2y, alias_2y.c.date == alias_10y.c.date).where(alias_10y.c.date > datetime(2016, 1, 1))

In [42]:
with Session(engine) as session:
    # sql = select(alias_10y.c.date,alias_10y.c.rate-alias_2y.c.rate).join_from(alias_10y, alias_2y, alias_2y.c.date == alias_10y.c.date).compile(engine, compile_kwargs={"literal_binds": True})
    first_row = session.execute(select(alias_10y.c.date,cast(alias_10y.c.rate-alias_2y.c.rate, Numeric(10,2)).label("2s10s")).join_from(alias_10y, alias_2y, alias_2y.c.date == alias_10y.c.date).where(alias_10y.c.date > datetime(2016, 1, 1))).all()
    

In [48]:
first_row[0][1]

1.22

In [35]:
with Session(engine) as session:
    # sql = select(alias_10y.c.date,alias_10y.c.rate-alias_2y.c.rate).join_from(alias_10y, alias_2y, alias_2y.c.date == alias_10y.c.date).compile(engine, compile_kwargs={"literal_binds": True})
    print(session.execute(select(alias_10y).where(alias_10y.c.date > datetime(2022, 4, 1))).all())
    

[(datetime.date(2022, 4, 11), 2.79), (datetime.date(2022, 4, 8), 2.72), (datetime.date(2022, 4, 7), 2.66), (datetime.date(2022, 4, 6), 2.61), (datetime.date(2022, 4, 5), 2.54), (datetime.date(2022, 4, 4), 2.42)]


In [73]:
url = "https://www.alphavantage.co/query"
payload = {
    "function" : "TREASURY_YIELD",
    "maturity" :  '1y',
    # 'keywords': 'microsoft',
    "apikey": os.environ['ALPHA_VANTAGE']
}

r = requests.get(url, params=payload)

In [74]:
output = json.loads(r.text)

In [16]:
session.new

IdentitySet([])

In [37]:
session.commit()

In [30]:
len(output['data'])

11780

In [64]:
wb.economy.

APIResponseError: APIError: JSON decoding error (https://api.worldbank.org/v2/en/sources/2/country/SL.UEM.TOTL.ZS?per_page=1000&page=1&format=json)

In [69]:
vals = []
for ix in wb.data.fetch("SL.UEM.TOTL.ZS", economy = "USA"):
    vals.append(ix)

In [70]:
vals

[{'value': 8.05000019073486,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2020'},
 {'value': 3.67000007629395,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2019'},
 {'value': 3.90000009536743,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2018'},
 {'value': 4.3600001335144,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2017'},
 {'value': 4.86999988555908,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2016'},
 {'value': 5.28000020980835,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2015'},
 {'value': 6.17000007629395,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2014'},
 {'value': 7.36999988555908,
  'series': 'SL.UEM.TOTL.ZS',
  'economy': 'USA',
  'aggregate': False,
  'time': 'YR2013'},
 {'value': 8.069999694824

In [96]:
ix.to_pydatetime()

datetime.datetime(2022, 4, 14, 0, 0)

In [54]:
all_sources = wb.source.info()

In [60]:
all_sources

id,name,code,concepts,lastupdated
1,Doing Business,DBS,3,2021-08-18
2,World Development Indicators,WDI,3,2022-04-08
3,Worldwide Governance Indicators,WGI,3,2021-09-27
5,Subnational Malnutrition Database,SNM,3,2016-03-21
6,International Debt Statistics,IDS,4,2022-01-14
11,Africa Development Indicators,ADI,3,2013-02-22
12,Education Statistics,EDS,3,2020-12-20
13,Enterprise Surveys,ESY,3,2022-03-25
14,Gender Statistics,GDS,3,2022-03-01
15,Global Economic Monitor,GEM,3,2020-07-27


In [56]:
all_sources['Name']

TypeError: 'Featureset' object is not subscriptable

In [53]:
wb.series.info(db=1)       

id,value
ENF.CONT.COEN.ATDR,Enforcing contracts: Alternative dispute resolution (0-3) (DB16-20 methodology)
ENF.CONT.COEN.ATFE.PR,Enforcing contracts: Attorney fees (% of claim)
ENF.CONT.COEN.COST.ZS,Enforcing contracts: Cost (% of claim)
ENF.CONT.COEN.COST.ZS.DFRN,Enforcing contracts: Cost (% of claim) - Score
ENF.CONT.COEN.CSMG,Enforcing contracts: Case management (0-6) (DB16-20 methodology)
ENF.CONT.COEN.CTAU,Enforcing contracts: Court automation (0-4) (DB17-20 methodology)
ENF.CONT.COEN.CTFE.PR,Enforcing contracts: Court fees (% of claim)
ENF.CONT.COEN.CTSP.DB16,Enforcing contracts: Court structure and proceedings (0-5) (DB16 methodology)
ENF.CONT.COEN.CTSP.DB1719,Enforcing contracts: Court structure and proceedings (0-5) (DB17-20 methodology)
ENF.CONT.COEN.DB0415.DFRN,Enforcing contracts (DB04-15 methodology) - Score


In [125]:
ty_1y.dropna().iloc[20:40]

1959-08-12    4.19
1959-08-13    4.19
1959-08-14    4.19
1959-08-17    4.25
1959-08-18    4.34
1959-08-19    4.32
1959-08-20    4.32
1959-08-21    4.45
1959-08-24    4.50
1959-08-25    4.46
1959-08-26    4.41
1959-08-27    4.40
1959-08-28    4.45
1959-08-31    4.60
1959-09-01    4.64
1959-09-02    4.68
1959-09-03    4.67
1959-09-04    4.62
1959-09-08    4.68
1959-09-09    4.68
dtype: float64

In [134]:
with Session(engine) as session:
    for ix, val in fred.get_series("DTB4WK").dropna().items():
        try:
            dt = ix.to_pydatetime()
            session.add(Treasury_Yield(maturity = '1m', currency = 'USD', date = dt , rate = float(val)))
            # newVal = float(val)
        except:
            print(ix, val)
    # session.flush()
    session.commit()